In [1]:
from utils_clean import *
from test_dataset_utils import *

In [ ]:

#download_patents_pto(start_year=2015, end_year=2015, kind='grant',download_path="data")
#unzip_files("data","patent_grants_2015")

In [ ]:
# # Example usage
# test_dataset = create_test_dataset_from_freilich(
#     year=2015,  # The year you want to analyze
#     freilich_data_path="Freilich.Data.Compressed.xlsb",  # Path to your Freilich dataset
#     path_to_all_xmls_for_chosen_year="patent_grants_2015"  # Directory containing XML files
# )

# # Optional: Check the results
# print(f"Number of patents extracted: {len(test_dataset)}")
# print("Sample document numbers:", list(test_dataset.keys())[:5])

Processing ipg150106.xml
Processing ipg150113.xml
Found 869 out of 25081 documents
Saved as test_dataset_2015.json
Number of patents extracted: 869
Sample document numbers: ['RE045323', 'RE045324', 'RE045325', '8925349', '8925551']


In [2]:
test_data = load_from_pickle("test_dataset_2015.pkl")

df_test_data = pd.DataFrame(test_data,index=["xml"],).T.reset_index()
df_test_data.columns = ["patentnumber","xml"] 

Loaded 25081 patents from test_dataset_2015.pkl


In [3]:
# df = read_xlsb_file()
# df[(df.issueyear == 2015) | (df.prophetic !=0)][["patentnumber","prophetic","nonprophetic","issueyear"]]
# df[df.issueyear == 2015].to_csv("freilichdataet_2015.csv")
df = pd.read_csv("freilichdataet_2015.csv")
df["patentnumber"] = df["patentnumber"].astype(str).transform(lambda x: x.replace(".0", ""))
df["patentnumber"] = df["patentnumber"].apply(remove_leadiong_zeros)
df.head()

,Unnamed: 0,patentnumber,appyear,issueyear,inventorcountry,divisional,continuation,prioritydate,priority.formatted,priorityyear,...,yr4,yr8,yr11,claims,wordaverage,litigated,orangebook,forward.cites,industry,subindustry
0,497653,8969641,1976.0,2015.0,US,0.0,0.0,19760903.0,28006.0,1976.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,19.0
1,497654,9134302,1995.0,2015.0,US,1.0,1.0,19830505.0,30441.0,1983.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN
2,497655,8926966,1995.0,2015.0,US,1.0,1.0,19900921.0,33137.0,1990.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,3.0,31.0
3,497656,9181140,1993.0,2015.0,US,0.0,1.0,19920916.0,33863.0,1992.0,...,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN
4,497657,9199886,2009.0,2015.0,US,0.0,1.0,19940119.0,34353.0,1994.0,...,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN


In [4]:
merged = df_test_data.merge(df, on="patentnumber", how="left")
print(len(merged))
cols = ["patentnumber","xml","prophetic","nonprophetic","allprophetic","someprophetic"]
merged[cols].head()

25081


,patentnumber,xml,prophetic,nonprophetic,allprophetic,someprophetic
0,RE045323,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT...",1.0,12.0,0.0,1.0
1,RE045324,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT...",0.0,8.0,0.0,0.0
2,RE045325,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT...",0.0,8.0,0.0,0.0
3,8925349,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT...",26.0,0.0,1.0,0.0
4,8925551,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT...",6.0,5.0,0.0,1.0


In [6]:

doc_w_exp = {}
i = 0
for i,xml in enumerate(merged["xml"]):
    i += 1
    if i % 1000 == 0:
        print(f"{i}/{len(merged.xml)} so far found {len(doc_w_exp)} docs with experiments")
    heading = extract_experiments_w_heading(xml)
    if heading:
        extracted_examples = []
        if len(heading) >1:
            print(f"has more than 1 Examples heading , {i}")
            # for example in heading:
            #     extracted_ex_start_w_word = process_siblings(example.find_next_siblings())
            #     if extracted_ex_start_w_word and len(extracted_ex_start_w_word) > 0:
            #         extracted_examples.append(extracted_ex_start_w_word)
            #     else:
            #         extracted_ex_w_word = extract_examples_start_w_word(example.find_next_siblings())
            #         if extracted_ex_w_word:
            #             if len(extracted_ex_w_word) > 0:
            #                 extracted_examples.append(extracted_ex_w_word)
            #         else:
            #             num_dot_examples = extract_num_dot_examples(str(heading[0].find_next_siblings()))
            #             if len(num_dot_examples) > 0:
            #                 extracted_examples.append(num_dot_examples)
        elif len(heading) == 1:
            example_start_w = process_siblings(heading[0].find_next_siblings())
            if example_start_w:
                if len(example_start_w[0]["content"]) == 0:
                    extracted_ex_w_word = extract_examples_start_w_word(heading[0].find_next_siblings())
                    try:
                        if len(extracted_ex_w_word["content"]) > 0:
                            extracted_examples.append(extracted_ex_w_word)
                    except:
                        if len(extracted_ex_w_word[0]["content"]) > 0:
                            extracted_examples.append(extracted_ex_w_word)
                else:
                    extracted_examples.append(example_start_w)
            else:
                extracted_ex_w_word = extract_examples_start_w_word(heading[0].find_next_siblings())
                if extracted_ex_w_word:
                    if len(extracted_ex_w_word["content"]) > 0:
                        extracted_examples.append(extracted_ex_w_word)
                else:
                    num_dot_examples = extract_num_dot_examples(str(heading[0].find_next_siblings()))
                    if len(num_dot_examples) > 0:
                        extracted_examples.append(num_dot_examples)
        else:
            extracted_ex_w_word = extract_examples_w_word(xml)
            if extracted_ex_w_word:
                if len(extracted_ex_w_word["content"]) > 0:
                    extracted_examples.append(extracted_ex_w_word)
                else:
                    example_start_w = process_siblings(heading[0].find_next_siblings())
                    if example_start_w:
                        if len(example_start_w[0]["content"]) > 0:
                            extracted_examples.append(example_start_w)

            # else:
            #     num_dot_examples = extract_num_dot_examples(str(heading[0].find_next_siblings()))
            #     extracted_examples.append(num_dot_examples)
    
        
    
        if len(extracted_examples) > 0:
            doc_w_exp[find_doc_number(xml)[0]] = extracted_examples
            # if len(doc_w_exp) == 100:
            #     break

KeyboardInterrupt: 

In [5]:
def extract_examples_from_heading(heading):
    extracted_examples = []
    example_start_w = process_siblings(heading.find_next_siblings())
    if example_start_w:
        if not example_start_w[0]["content"]:
            extracted_ex_w_word = extract_examples_start_w_word(heading.find_next_siblings())
            if extracted_ex_w_word:
                if isinstance(extracted_ex_w_word, list):
                    if extracted_ex_w_word and extracted_ex_w_word[0]["content"]:
                        extracted_examples.append(extracted_ex_w_word)
                elif extracted_ex_w_word["content"]:
                    extracted_examples.append(extracted_ex_w_word)
        else:
            extracted_examples.append(example_start_w)
    else:
        extracted_ex_w_word = extract_examples_start_w_word(heading.find_next_siblings())
        if extracted_ex_w_word:
            if isinstance(extracted_ex_w_word, list):
                if extracted_ex_w_word and extracted_ex_w_word[0]["content"]:
                    extracted_examples.append(extracted_ex_w_word)
            elif extracted_ex_w_word["content"]:
                extracted_examples.append(extracted_ex_w_word)
        else:
            num_dot_examples = extract_num_dot_examples(str(heading.find_next_siblings()))
            if num_dot_examples:
                extracted_examples.append(num_dot_examples)
    return extracted_examples

def extract_examples(xml):
    heading = extract_experiments_w_heading(xml)
    if heading:
        if len(heading) > 1:
            print(f"has more than 1 Examples heading, {i}")
            return []
        elif len(heading) == 1:
            return extract_examples_from_heading(heading[0])
        else:
            extracted_ex_w_word = extract_examples_w_word(xml)
            if extracted_ex_w_word:
                if isinstance(extracted_ex_w_word, list):
                    if extracted_ex_w_word and extracted_ex_w_word[0]["content"]:
                        return [extracted_ex_w_word]
                elif extracted_ex_w_word["content"]:
                    return [extracted_ex_w_word]
            else:
                example_start_w = process_siblings(heading[0].find_next_siblings())
                if example_start_w and example_start_w[0]["content"]:
                    return [example_start_w]
    return []

doc_w_exp = {}
for i, xml in enumerate(merged["xml"][:4000], start=1):
    if i % 1000 == 0:
        print(f"{i}/{len(merged.xml)} so far found {len(doc_w_exp)} docs with experiments")
    
    extracted_examples = extract_examples(xml)
    if extracted_examples:
        doc_w_exp[find_doc_number(xml)[0]] = extracted_examples

has more than 1 Examples heading, 349
has more than 1 Examples heading, 388
has more than 1 Examples heading, 689
has more than 1 Examples heading, 768
has more than 1 Examples heading, 781
has more than 1 Examples heading, 814
1000/25081 so far found 509 docs with experiments
has more than 1 Examples heading, 1054
has more than 1 Examples heading, 1417
has more than 1 Examples heading, 1475
has more than 1 Examples heading, 1810
2000/25081 so far found 1004 docs with experiments
has more than 1 Examples heading, 2065
has more than 1 Examples heading, 2131
has more than 1 Examples heading, 2454
has more than 1 Examples heading, 2470
has more than 1 Examples heading, 2562
has more than 1 Examples heading, 2636
has more than 1 Examples heading, 2749
has more than 1 Examples heading, 2774
3000/25081 so far found 1513 docs with experiments
has more than 1 Examples heading, 3523
has more than 1 Examples heading, 3545
has more than 1 Examples heading, 3976
4000/25081 so far found 2023 docs w

In [6]:
# save_as_json(doc_w_exp, "doc_w_exp_2015.json")
doc_w_exp = read_json("doc_w_exp_2015.json")

In [6]:
import re
import html
import unicodedata
from bs4 import BeautifulSoup

def clean_text_updated(text):
    """
    Clean text by removing HTML tags, special characters, extra spaces,
    and normalizing the content while keeping meaningful punctuation.

    Args:
        text (str): Input text to clean

    Returns:
        str: Cleaned text
    """
    if not isinstance(text, str):
        return ""

    # Decode HTML entities & remove HTML tags
    text = html.unescape(text)
    text = BeautifulSoup(text, "html.parser").get_text()

    # Normalize Unicode characters (e.g., é → e)
    text = unicodedata.normalize("NFKC", text)

    # Remove unwanted special characters but keep punctuation
    text = re.sub(r"[^\w\s.,!?'\-]", "", text)

    # Normalize spaces: remove multiple spaces, newlines, and tabs
    text = re.sub(r"\s+", " ", text).strip()

    return text

import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

def check_tense_nltk_updated(sentence):
    words = word_tokenize(sentence)
    tagged = pos_tag(words)

    # Define tense categories
    past = {"VBD", "VBN"}
    present = {"VB", "VBG", "VBP", "VBZ"}
    future = {"MD"}

    tenses = {"past": 0, "present": 0, "future": 0}

    for i, (word, tag) in enumerate(tagged):
        # Count past tense words
        if tag in past:
            tenses["past"] += 1

        # Count present tense words
        elif tag in present:
            tenses["present"] += 1

        # Future tense handling
        elif tag in future:
            if word.lower() in {"will", "shall"}:
                # Ensure 'will' or 'shall' is followed by a verb (to confirm future tense)
                if i + 1 < len(tagged) and tagged[i + 1][1] in {"VB", "VBP"}:
                    tenses["future"] += 1

    return max(tenses, key=tenses.get) if max(tenses.values()) > 0 else "Unknown"



In [7]:
import re
from nlp_processing import check_tense_nltk,clean_text

def dic_to_dic_w_tense(doc_w_exp):
    dic = {}
    pattern = r'\(\d+\)\s*([A-Za-z0-9\-\(\)\{\},:;=\[\]\+\*\s\.\^\$\%]+(?:\.(?:sup|delta|Hz|NMR)[^\)]*)?)'
    pattern2 = r'\(\d+\)\s*(?:[A-Za-z]+\s*\d*\-?[A-Za-z]*[\(\{][^)]*[\)\}][^;]*|(?:\.\d*H\-?NMR[^;]*|\d+\.[a-zA-Z]*\d*))'


    for key, value in doc_w_exp.items():
        tense_counts = {"past": 0, "present": 0,"Unknown":0}
        
        if isinstance(value[0], list) and len(value[0]) == 1:
            desc = value[0][0]["title"] + "." + "".join(value[0][0]["content"])
            tense = check_tense_nltk_updated(clean_text(desc))
            if tense !=  "Unknown":
                tense_counts[tense] += 1
            else:
                matches = re.findall(pattern, desc)
                if matches:
                    tense_counts["past"] += 1
                else:
                    tense_counts["Unknown"] += 1
            dic[key] = tense_counts


        elif isinstance(value[0], list) and len(value[0]) > 1:
            for ls in value[0]:
                desc = ls["title"] + "." + "".join(ls["content"])
                tense = check_tense_nltk_updated(clean_text(desc))
                if tense !=  "Unknown":
                    tense_counts[tense] += 1
                else:
                    matches = re.findall(pattern, desc)
                    if matches:
                        tense_counts["past"] += 1
                    else:
                        tense_counts["Unknown"] += 1
            dic[key] = tense_counts

        elif isinstance(value[0], dict):
            for ex, description in value[0].items():
                tense = check_tense_nltk_updated(clean_text(description))
                if tense !=  "Unknown":
                    tense_counts[tense] += 1
                else:
                    matches = re.findall(pattern, description)
                    if matches:
                        tense_counts["past"] += 1
                    else:
                        tense_counts["Unknown"] += 1
            dic[key] = tense_counts

    return dic

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\AM3081\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\AM3081\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
list_w_exp = list(doc_w_exp.items())[:2000]
final = dic_to_dic_w_tense(dict(list_w_exp))
df_final = pd.DataFrame(final).T.reset_index()
df_final.columns = ["patentnumber","past","present","Unknown"]
df_final["patentnumber"] = df_final["patentnumber"].apply(remove_leadiong_zeros)

c:\ProgramData\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [10]:
import numpy as np

df_check = df_final.merge(merged,on= "patentnumber",how="left")[["patentnumber","present","past","Unknown","prophetic","nonprophetic","allprophetic","someprophetic"]]
#df_check["past"] = df_check["past"] + df_check["Unknown"]
df_check["Total_Extracted"] = df_check["past"] + df_check["present"] + df_check["Unknown"]
df_check["Total_Freilich"] = df_check["prophetic"] + df_check["nonprophetic"]
df_check["prophetic_error"] = np.sqrt((df_check["prophetic"] - df_check["present"])**2)
df_check["nonprophetic_error"] = np.sqrt((df_check["nonprophetic"] - df_check["past"])**2)
df_check["Total_Mean_error"] = np.sqrt((df_check["Total_Freilich"] - df_check["Total_Extracted"])**2)
df_check["Sum_error"] = df_check["prophetic_error"] + df_check["nonprophetic_error"] + df_check["Total_Mean_error"]
print(f"Number of exact matches: {len(df_check[(df_check.prophetic_error ==0) & (df_check.nonprophetic_error ==0) & (df_check.Total_Mean_error ==0)])} out of {len(df_check)} , Percentage: {len(df_check[(df_check.prophetic_error ==0) & (df_check.nonprophetic_error ==0) & (df_check.Total_Mean_error ==0)])/len(df_check)*100}")
print(f"Number of exact num of patent extracted: {len(df_check[(df_check.Total_Extracted == df_check.Total_Freilich)])} out of {len(df_check)}")
print(f"Avg Total error: {df_check['Total_Mean_error'].mean()}, num of corrects:  {df_check[df_check['Total_Mean_error'] == 0].shape[0]}")
print(f"Avg Total prophetic error: {df_check['prophetic_error'].mean()}, num of corrects: {df_check[df_check['prophetic_error'] == 0].shape[0]}")
print(f"Avg Total nonprophetic error: {df_check['nonprophetic_error'].mean()}, num of corrects: {df_check[df_check['nonprophetic_error'] == 0].shape[0]}")
print(f"{df_check['Sum_error'].sum()}")
df_check.sort_values("Total_Mean_error",ascending=False).head(20)



#With clean_text and tense updated description for first 2000
# Number of exact matches: 626 out of 2000 , Percentage: 31.3
# Number of exact num of patent extracted: 1254 out of 2000
# Avg Total error: 2.1615, num of corrects:  1254
# Avg Total prophetic error: 1.4895, num of corrects: 1041
# Avg Total nonprophetic error: 3.5455, num of corrects: 733
# 14393.0


# not extracting reference examples
# Number of exact matches: 641 out of 2000 , Percentage: 32.05
# Number of exact num of patent extracted: 1274 out of 2000
# Avg Total error: 1.717, num of corrects:  1274
# Avg Total prophetic error: 1.483, num of corrects: 1045
# Avg Total nonprophetic error: 3.126, num of corrects: 750
# 12652.0


# not extracting reference and preparation examples
# Number of exact matches: 650 out of 2000 , Percentage: 32.5
# Number of exact num of patent extracted: 1293 out of 2000
# Avg Total error: 1.561,f num of corrects:  1293
# Avg Total prophetic error: 1.4595, num of corrects: 1049
# Avg Total nonprophetic error: 3.028, num of corrects: 751
# 12097.0




Number of exact matches: 650 out of 2000 , Percentage: 32.5
Number of exact num of patent extracted: 1289 out of 2000
Avg Total error: 1.5815, num of corrects:  1289
Avg Total prophetic error: 1.48, num of corrects: 1044
Avg Total nonprophetic error: 3.031, num of corrects: 757
12185.0


,patentnumber,present,past,Unknown,prophetic,nonprophetic,allprophetic,someprophetic,Total_Extracted,Total_Freilich,prophetic_error,nonprophetic_error,Total_Mean_error,Sum_error
366,8933099,0,131,0,0.0,33.0,0.0,0.0,131,33.0,0.0,98.0,98.0,196.0
0,RE045323,3,42,12,1.0,12.0,0.0,1.0,57,13.0,2.0,30.0,44.0,76.0
655,8940412,3,52,0,0.0,12.0,0.0,0.0,55,12.0,3.0,40.0,43.0,86.0
83,8927118,0,41,0,0.0,4.0,0.0,0.0,41,4.0,0.0,37.0,37.0,74.0
486,8936854,1,33,1,0.0,1.0,0.0,0.0,35,1.0,1.0,32.0,34.0,67.0
868,8945701,40,18,0,27.0,0.0,1.0,0.0,58,27.0,13.0,18.0,31.0,62.0
5,8926083,1,78,0,0.0,49.0,0.0,0.0,79,49.0,1.0,29.0,30.0,60.0
193,8927721,12,8,11,0.0,1.0,0.0,0.0,31,1.0,12.0,7.0,30.0,49.0
1386,8957140,0,30,0,0.0,1.0,0.0,0.0,30,1.0,0.0,29.0,29.0,58.0
1275,8956552,1,17,22,0.0,12.0,0.0,0.0,40,12.0,1.0,5.0,28.0,34.0


In [11]:
doc_w_exp["08933099"]

[[{'number': 'Example 1',
   'title': '6-Methoxy-N-methyl-5-((2-(4-(piperidin-4-yl)benzamide)pyridin-4-yl)oxy)-1H-indole-1-carboxamide',
   'content': ['',
    'tert-Butyl 4-(4-((4-((6-methoxy-1-(methylcarbamoyl)-1H-indol-5-yl)oxy)pyridin-2-yl)carbamoyl)phenyl)piperidine-1-carboxylate described in Production Example 1-8 (3.42 g, 5.70 mmol) was dissolved in dichloromethane (45 mL), and trifluoroacetic acid (15 mL) was added at 0° C. The mixture was stirred at room temperature for 60 minutes and then concentrated under vacuum, and then the residue was dissolved in dichloromethane-triethylamine, and the resultant was purified with NH silica gel column chromatography (ethyl acetate:methanol=97:3-4:1). The target fraction was concentrated under vacuum and then the precipitate was collected by filteration and washed with diethyl ether and ethyl acetate to obtain the title compound (2.61 g, 92%).',
    '1H-NMR Spectrum (CDCl3) δ (ppm): 1.55-1.70 (2H, m), 1.78-1.88 (2H, m), 2.62-2.80 (3H, m), 

# ERRORS IN FREILICH DATASET:
patentnumber 8957069: in freilich 108.0	nonproph and 0 prophetic but there are many more examples than 108, our algorithm: 267 nonprophetic and 1 prophetic and 3 unknown
patent number 8946443:we extracted 271	freilich: 108, this patent has refrtence examples, are we extracting these or not? 
patent number 8952010: same issue do we take refrence examples? 
patent 8933099: there are far more than 33 examples in this patent but freilich says 33, our algorithm found 131
what to do with Preparation examples?? e.g 8962612 in freilich has 6 we got 70.

# improvement for our algorithm
1. remove duplicates somehow, e.g in patent number 08987242, there some examples are extracted twice just because the name e.g example aa137 appears multiple times.
2. better classification of the patents. e.g patent 8987295 has 442, we extract the same number as frelich but we clasify differently.
3. 

In [92]:
check_tense_nltk("""9-(4-(1-Aminoethyl)phenyl)-6-chloro-8-hydroxythieno[2,3-c]quinolin-4(5H)-o- ne Hydrochloride
(573) ##STR00933##
(574) Following General Procedure F, tert-butyl 1-(4-(6-chloro-8-methoxy-4-oxo-4,5-dihydrothieno[2,3-c]quinolin-9-yl)phen- yl)ethylcarbamate (50 mg, 0.10 mmol) was treated with BBr.sub.3 (1.0 M in CH.sub.2Cl.sub.2, 2 mL, 2 mmol) to afford the desired product (21 mg, 58%) as a light yellow solid (21 mg, 58%): .sup.1H NMR (500 MHz, CD.sub.4OD) .delta. 7.65 (dt, J=5.2, 3.4 Hz, 2H), 7.59 (d, J=5.4 Hz, 1H), 7.41 (dt, J=4.0, 2.6 Hz, 2H), 7.30 (s, 1H), 6.07 (d, J=: 5.4 Hz, 1H), 4.62 (q, J=6.8 Hz, 1H), 1.76 (d, J=6.9 Hz, 3H); ESI MS m/z 371 [C.sub.19H.sub.15ClN.sub.1O.sub.2S+H].sup.+; HPLC 97.8% (AUC), t.sub.R=9.72 min.""")

'past'

In [83]:
df_check[df_check.Unknown > 0]

,patentnumber,present,past,Unknown,prophetic,nonprophetic,allprophetic,someprophetic,Total_Extracted,Total_Freilich,prophetic_error,nonprophetic_error,Total_Mean_error
12,8946423,1,9,1,0.0,9.0,0.0,0.0,11,9.0,1.0,0.0,2.0
16,8946430,4,38,11,0.0,53.0,0.0,0.0,53,53.0,4.0,15.0,0.0
17,8946434,3,23,3,3.0,23.0,0.0,1.0,29,26.0,0.0,0.0,3.0
18,8946437,8,19,34,0.0,40.0,0.0,0.0,61,40.0,8.0,21.0,21.0


In [35]:
merged[merged.patentnumber == "8926083"][cols]

,patentnumber,xml,prophetic,nonprophetic,allprophetic,someprophetic
8,8926083,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT...",0.0,49.0,0.0,0.0
